# LTE Data Science Capstone Project
Develop a machine learning model to analyze and predict car accident severity


In [2]:
import pandas as pd
import numpy as np

print(f'Hello Capstone Project Course!')

Hello Capstone Project Course!


## Introduction: Business Understanding

The negative consequences of road traffic related deaths cannot be understated. According to a February 2020 report from the World Health Organization, road traffic crashes account for over 1.3 million deaths each year, costs countries as much as 3% of their GDP annually, and represent the leading cause of death for those aged 5-29 years. Furthermore, its not just the drivers and their families who are impacted – over half the deaths associated with road traffic collisions are attributed to non-driving road users: pedestrians, cyclists and motorcyclists (https://www.who.int/news-room/fact-sheets/detail/road-traffic-injuries). In the United States, according to data from the National Highway Traffic Safety Administration (NHTSA), 2018 recorded over 36,000 deaths from traffic accidents and over 6.7 million total crashes causing property damage and loss through opportunity cost (https://www.iii.org/fact-statistic/facts-statistics-highway-safety). The problem is multi-faceted; however it can ultimately come down to a driver’s complicity and attention deficit when routinely driving in a diverse set of road conditions. The need to address this problem will be to raise drivers’ awareness to the increased risk they take when driving in adverse conditions – rain, low visibility, etc. 

It does not have to be this way. Data being compiled by public and private agencies are able to be put to use in new ways and generating tools to reduce the number and impact of traffic collisions. The rising availability of technology and machine learning techniques are being developed and deployed into communities aimed at informing motorists and redirecting their driving activities to safer options when information and statistics indicate a risk to do otherwise. With the need to reduce car collisions in an area, this paper proposes a model to indicate the severity of an accident given the current observable conditions. When these conditions are adverse, this model will alert drivers of this to improve their awareness or elect to find another mode of transportation if required. 


## Data Understanding

For a proof of concept, this model will be based on Seattle, WA data and leverages road, weather, and visibility conditions to indicate the severity of a potential accident. Our dependent/predictor variable will be 'SEVERITYCODE' because it is used measure the severity of an accident from 0 to 5 within the dataset. For the purposes of this paper and given dataset, the independent variables to be used are 'WEATHER', 'ROADCOND' and 'LIGHTCOND' as these are consistently populated throughout the dataset and arguably have the greatest impact on a vehicle and driver’s performance.